# 1

In [492]:
#gym
import gymnasium as gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

#helpers
import numpy as np
import random
import os

#stableblines
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [494]:
import torch

In [496]:
torch.cuda.is_available()

True

# 2

In [499]:
Discrete(3).sample()

2

In [501]:
Box(0,1, shape=(3,3)).sample()

array([[0.19899589, 0.06866928, 0.17509998],
       [0.7751671 , 0.54645056, 0.5902371 ],
       [0.759381  , 0.14563976, 0.29931268]], dtype=float32)

In [503]:
Tuple((Discrete(3), Box(0,1, shape=(3,)))).sample()

(1, array([0.76736337, 0.50837094, 0.93968165], dtype=float32))

In [505]:
Dict({'height':Discrete(6), 'speed':Box(0,100, shape=(1,))}).sample()

OrderedDict([('height', 3), ('speed', array([14.996329], dtype=float32))])

In [507]:
MultiBinary(4).sample()

array([0, 0, 0, 0], dtype=int8)

In [509]:
MultiDiscrete([5,2,2]).sample()

array([4, 1, 0], dtype=int64)

# 3

In [512]:
Box(low=np.array([0]), high=np.array([100]))

Box(0.0, 100.0, (1,), float32)

In [514]:
Box(0,100, shape=(1,))

Box(0.0, 100.0, (1,), float32)

In [516]:
np.array([38+random.randint(-3,3)]).astype(float)

array([37.])

In [562]:
import time

In [611]:
class ShowerEnv(Env):
    def __init__(self): #sets up the env when first run?
        self.action_space = Discrete(3) # tap up, down, or no action
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        self.state = 38 + random.randint(-3,3)
        self.shower_length = 60
        
    def step(self, action): #
        #apply temp
        self.state += action-1

        #decreate time
        self.shower_length -= 1

        #calc reward
        if self.state >= 37 and self.state <=39:
            reward = 1
        else:
            reward = -1

        if self.shower_length <=0:
            done = True
        else:
            done = False

        info = {}

        time.sleep(0.1)
        return self.state, reward, done, info

    def render(self): #renders env
        #not going to implement this
        pass
    def reset(self): #resets temp and shower time
        self.state = 38 + random.randint(-3,3)#idk why tf but u can do this too??? np.array([38+random.randint(-3,3)]).astype(float)
        self.shower_length = 60
        return self.state

In [613]:
env = ShowerEnv()

In [615]:
env.observation_space

Box(0.0, 100.0, (1,), float32)

In [617]:
env.action_space

Discrete(3)

In [619]:
env.reset()

37

In [621]:
env.step(2)

(38, 1, False, {})

# 4

In [624]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-4


KeyboardInterrupt: 

# 5

In [626]:
log_path = os.path.join('Training', 'Logs')
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [628]:
model.learn(total_timesteps=10000)

Logging to Training\Logs\PPO_11
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -28.1    |
| time/              |          |
|    fps             | 9        |
|    iterations      | 1        |
|    time_elapsed    | 213      |
|    total_timesteps | 2048     |
---------------------------------


KeyboardInterrupt: 

# 6

In [87]:
shower_path = os.path.join('Training', 'Saved Models', 'Shower_Model_PPO')

In [88]:
model.save(shower_path)

In [89]:
del model

In [91]:
model = PPO.load(shower_path, env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


C:\Users\nurma\TestEnvOne\EnvOne\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [548]:
evaluate_policy(model, env, n_eval_episodes=200)

C:\Users\nurma\anaconda3\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(-10.2, 59.12664374036463)